# Supertrend ontwikkeling en berekening voor nieuwe strategie

Bron: https://www.youtube.com/watch?v=JqEjhBD55NY

Daarna Pandas-ta gebruikt en deze bron nagebouwd: https://www.youtube.com/watch?v=6PbwVZ6HwRE

In [57]:
# Importeer de benodigde bibliotheken
import pandas as pd
import pandas_ta as pta
from freqtrade.strategy.interface import IStrategy
from pandas import DataFrame
import talib.abstract as ta
import freqtrade.vendor.qtpylib.indicators as qtpylib
from scipy.spatial.distance import cosine
import numpy as np

In [58]:
# bij weergave van een dataframe is de standaard weergave enkele regels van het begin df en enkele regels van het einde df. Met deze optie laat panda's echt alle regels in de dataframe zien.
pd.set_option('display.max_rows', None)
# Turn off copy in dataframe warnings
#df.is_copy = False
# pd.set_option('df.is_copy' = False)
# pd.set_option('mode.chained_assignment', 'warn')
# if you set a value on a copy, warning will show

In [59]:
# Load data
df = pd.read_json("ETH_USDT-1d.json")
df.columns=['date','open','high','low','close','volume']
df['date']=(pd.to_datetime(df['date'],unit='ms'))
df.tail(5)

,date,open,high,low,close,volume
994,2021-05-28,2742.23,2762.45,2327.38,2412.10,1.554585e+06
995,2021-05-29,2412.34,2574.27,2204.10,2278.74,1.379427e+06
996,2021-05-30,2278.74,2476.42,2180.53,2385.98,1.151113e+06
997,2021-05-31,2385.82,2720.00,2271.44,2706.15,1.307958e+06
998,2021-06-01,2706.15,2740.00,2523.74,2634.57,1.179577e+06


In [60]:
def supertrend(df, period=7,multiplier=3):
    # Create values for supertrend calculation
    hl2 = ((df['high'] + df['low'])/2)
    df['atr'] = ta.ATR(df, timeperiod=period)
    df['atr_upperband'] = hl2 + (multiplier*df['atr'])
    df['atr_lowerband'] = hl2 - (multiplier*df['atr'])
    df['upperband'] = hl2 + (multiplier*df['atr'])
    df['lowerband'] = hl2 - (multiplier*df['atr'])
    df['in_uptrend'] = True
    
    # Create the supertrend indicator
    for current in range(1, len(df.index)):
        previous = current - 1
        # check if the price is in an uptrend or downtrend
        if df['close'][current] > df['upperband'][previous]:
            # we are in an uptrend
            df['in_uptrend'][current] = True
        elif df['close'][current] < df['lowerband'][previous]:
            # we are in a downtrend
            df['in_uptrend'][current] = False
        else:
            # no change, so we stay in the current trend
            df['in_uptrend'][current] = df['in_uptrend'][previous]

            # if in an uptrend, move the lowerband up to the most recent higher lowerband value
            # (moves lowerband higher in uptrend)
            if df['in_uptrend'][current] and df['lowerband'][current] < df['lowerband'][previous]:
                df['lowerband'][current] = df['lowerband'][previous]
            # if not in an uptrend, move the upperband to the most recent lower upperband value
            # (moves upperband lower in a downtrend)
            if not df['in_uptrend'][current] and df['upperband'][current] > df['upperband'][previous]:
                df['upperband'][current] = df['upperband'][previous]
        
    return df

supertrend(df)

df[['date','open','high','low','close','atr','upperband','lowerband','in_uptrend']].tail(150)



TypeError: 'int' object is not callable

In [24]:
pta.supertrend(high=df['high'], low=df['low'], close=df['close'], period=7, multiplier=3).tail(250)

,SUPERT_7_3.0,SUPERTd_7_3.0,SUPERTl_7_3.0,SUPERTs_7_3.0
749,408.909787,-1,NaN,408.909787
750,408.909787,-1,NaN,408.909787
751,408.909787,-1,NaN,408.909787
752,408.909787,-1,NaN,408.909787
753,408.909787,-1,NaN,408.909787
754,408.909787,-1,NaN,408.909787
755,408.909787,-1,NaN,408.909787
756,404.184968,-1,NaN,404.184968
757,401.727115,-1,NaN,401.727115
758,400.298956,-1,NaN,400.298956


In [11]:
df['sup'] = pta.supertrend(high=df['high'], low=df['low'], close=df['close'], period=7, multiplier=3)['SUPERT_7_3.0']
df[['date','close','sup']].head(50)

,date,close,sup
0,2018-09-07,215.22,0.000000
1,2018-09-08,196.71,NaN
2,2018-09-09,195.81,NaN
3,2018-09-10,197.15,NaN
4,2018-09-11,185.18,NaN
5,2018-09-12,183.12,NaN
6,2018-09-13,210.68,127.485819
7,2018-09-14,208.82,143.179468
8,2018-09-15,221.29,149.381217
9,2018-09-16,220.17,151.640750


In [64]:
df['buy_signal'] = 0
df['sell_signal'] = 0
n = 6

# range will take a range from n until len(df), including n but not including df(len), insteps of one, 
# and iterate for each of the element in this range using for loop.
for i in range(n, len(df)):
    if (df['close'][i-1]<= df['sup'][i-1]) and (df['close'][i] > df['sup'][i]):
        df['buy_signal'][i] = 1
    if (df['close'][i-1]>= df['sup'][i-1]) and (df['close'][i] < df['sup'][i]):
        df['sell_signal'][i] = 1

# df[['date','close','sup','buy_signal','sell_signal']].head(10)
df.head(10)

TypeError: 'int' object is not callable

In [13]:
df[['date','close','sup','buy_signal','sell_signal']][(df['buy_signal'] > 0) | (df['sell_signal'] > 0)]

,date,close,sup,buy_signal,sell_signal
34,2018-10-11,191.58,249.987571,0,1
60,2018-11-06,220.74,191.220624,1,0
68,2018-11-14,187.43,224.249628,0,1
104,2018-12-20,114.05,77.496829,1,0
128,2019-01-13,114.16,151.356259,0,1
154,2019-02-08,119.25,92.704083,1,0
178,2019-03-04,125.65,152.781705,0,1
207,2019-04-02,163.31,131.896601,1,0
307,2019-07-11,268.63,343.671751,0,1
375,2019-09-17,207.84,175.860163,1,0


In [15]:
df['buysell'] = pta.supertrend(high=df['high'], low=df['low'], close=df['close'], period=7, multiplier=3)['SUPERTd_7_3.0']
df[['date','close','sup','buy_signal','sell_signal','buysell']][(df['buy_signal'] > 0) | (df['sell_signal'] > 0)]


,date,close,sup,buy_signal,sell_signal,buysell
34,2018-10-11,191.58,249.987571,0,1,-1
60,2018-11-06,220.74,191.220624,1,0,1
68,2018-11-14,187.43,224.249628,0,1,-1
104,2018-12-20,114.05,77.496829,1,0,1
128,2019-01-13,114.16,151.356259,0,1,-1
154,2019-02-08,119.25,92.704083,1,0,1
178,2019-03-04,125.65,152.781705,0,1,-1
207,2019-04-02,163.31,131.896601,1,0,1
307,2019-07-11,268.63,343.671751,0,1,-1
375,2019-09-17,207.84,175.860163,1,0,1


In [41]:
# Print eerste rij date
print(df['date'][1])

2018-09-08 00:00:00


In [42]:
# Print tweede rij date
print(df['date'][2])

2018-09-09 00:00:00


In [43]:
# Print vierde rij date
print(df['date'][4])

2018-09-11 00:00:00


In [44]:
# Print 4-1 is derde rij date
print(df['date'][4-1])

2018-09-10 00:00:00


In [49]:
# n = 6
# for i in range(n, len(df)):
#     print(1)
#     print(len(df))
length = len(df)
print(length)

TypeError: 'int' object is not callable